# Lactate Response
## Team 3 - Interventions in First Six Hours
### MIT Critical Data

The purpose of this notebook is to identify if the patient has had pressors, MV and RRT in the first 6 hours in the ICU.

## 0 - Envrionment

In [1]:
import numpy as np
import pandas as pd
import psycopg2

dbname = 'mimic'
schema_name = 'mimiciii'
db_schema = 'SET search_path TO {0};'.format(schema_name)
con = psycopg2.connect(database=dbname)

## 1 - Materialized Views

We generate the usual helper functions.

In [2]:
def execute_query_safely(sql, con):
    cur = con.cursor()
    try:
        cur.execute(sql)
    except:
        cur.execute('rollback;')
        raise
    finally:
        cur.close()
    return

def generate_materialized_view(query_file, con, query_schema):
    with open(query_file) as fp:
        query = ''.join(fp.readlines())
    print('Generating materialized view using {} ...'.format(query_file), end=' ')
    execute_query_safely(query_schema + query, con)
    print('done.')

Then we generate the views that have durations.

In [3]:
generate_materialized_view(query_file='./sql/ventilation-durations.sql', con=con, query_schema=db_schema)
generate_materialized_view(query_file='./sql/crrt-durations.sql', con=con, query_schema=db_schema)
generate_materialized_view(query_file='./sql/vasopressor-durations.sql', con=con, query_schema=db_schema)

Generating materialized view using ./sql/ventilation-durations.sql ... done.
Generating materialized view using ./sql/crrt-durations.sql ... done.
Generating materialized view using ./sql/vasopressor-durations.sql ... done.


Next, we craft a query for determining if these occured in the first 6 hours of the ICU stay.

## 2 - Query

We'll start with ventilation.

In [4]:
query = db_schema + '''

WITH vent_offset AS (
    SELECT ie.icustay_id, 
        ROUND((CAST(vd.starttime AS date) - CAST(ie.intime AS date))/365.242, 2) AS vent_offset
    FROM icustays ie
    LEFT JOIN ventdurations vd
        ON ie.icustay_id = vd.icustay_id
)
SELECT vo.icustay_id,
    MAX (CASE 
        WHEN vo.vent_offset <= 0.25 AND vo.vent_offset >= -0.83 THEN 1
        ELSE 0 END) AS vent_6hours
FROM vent_offset vo
GROUP BY vo.icustay_id;
'''

vent_df = pd.read_sql_query(query, con)
print(vent_df.shape)
vent_df.head()

(61532, 2)


,icustay_id,vent_6hours
0,233338,0
1,297372,0
2,243212,1
3,290736,0
4,262921,1


Now we apply the same approach to CRRT.

In [5]:
query = db_schema + '''

WITH crrt_offset AS (
    SELECT ie.icustay_id, 
        ROUND((CAST(cd.starttime AS date) - CAST(ie.intime AS date))/365.242, 2) AS crrt_offset
    FROM icustays ie
    LEFT JOIN crrtdurations cd
        ON ie.icustay_id = cd.icustay_id
)
SELECT co.icustay_id,
    MAX (CASE 
        WHEN co.crrt_offset <= 0.25 AND co.crrt_offset >= -0.83 THEN 1
        ELSE 0 END) AS crrt_6hours
FROM crrt_offset co
GROUP BY co.icustay_id;
'''

crrt_df = pd.read_sql_query(query, con)
print(crrt_df.shape)
crrt_df.head()

(61532, 2)


,icustay_id,crrt_6hours
0,233338,0
1,297372,0
2,243212,0
3,290736,0
4,262921,0


Finally, we turn to vasopressors.

In [6]:
query = db_schema + '''

WITH pressor_offset AS (
    SELECT ie.icustay_id, 
        ROUND((CAST(vpd.starttime AS date) - CAST(ie.intime AS date))/365.242, 2) AS pressor_offset
    FROM icustays ie
    LEFT JOIN vasopressordurations vpd
        ON ie.icustay_id = vpd.icustay_id
)
SELECT vp.icustay_id,
    MAX (CASE 
        WHEN vp.pressor_offset <= 0.25 THEN 1
        ELSE 0 END) AS vp_6hours
FROM pressor_offset vp
GROUP BY vp.icustay_id;
'''

pressor_df = pd.read_sql_query(query, con)
print(pressor_df.shape)
pressor_df.head()

(61532, 2)


,icustay_id,vp_6hours
0,233338,0
1,297372,0
2,243212,1
3,290736,1
4,262921,1


## 3 - Merge and Save

In [7]:
interventions_df = vent_df.merge(crrt_df)
interventions_df = interventions_df.merge(pressor_df)
interventions_df.to_csv('~/Desktop/interventions_6h.csv', index=False)